# Example Notebook

This notebook is designed demonstrate a number of goals:

  - The notebook is part of the automated analytical pipeline, as it is run programmatically by the build system, as in the dodo.py file.
  - It is tracked by version control via Git. To avoid large files and the problems associated with non-text files, the notebook is stripped of its output. 
  - In order to avoid re-running the notebook every time it changes (it changes often, even by the act of opening it) and to only rerun it if meaningful changes have been made, the build system only looks for changes in the plaintext version of the notebook. That is, the notebook is converted to a Python script via [nbconvert](https://nbconvert.readthedocs.io/en/latest/), which is often packaged with Jupyter.
  Then, DoIt looks for changes to the Python version. If it detects a difference, then the notebook is re-run. (Note, that you could also convert to a Markdown file with 
  [JupyText](https://github.com/mwouts/jupytext). However, this package is often not packaged with Jupyter.)
  - Since we want to use Jupyter Notebooks for exploratory reports, we want to keep fully-computed versions of the notebook (with the output intact). However, earlier I said that I strip the notebook of its output before committing to version control. Well, to keep the output, every time PyDoit runs the notebook, it outputs an HTML version of the freshly run notebook and saves that HTML report in the `output` directory. That way, you will be able to view the finished report at any time without having to open Jupyter.

import config
from pathlib import Path
OUTPUT_DIR = Path(config.OUTPUT_DIR)
DATA_DIR = Path(config.DATA_DIR)
WRDS_USERNAME = config.WRDS_USERNAME

START_DATE = config.START_DATE
END_DATE = config.END_DATE

import wrds

In [64]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path

import numpy as np
import pandas as pd
import wrds

from config import *
from load_nyu_call_report_data import *

In [65]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [66]:
DATA_DIR = Path(config.DATA_DIR)
pulled_dir = DATA_DIR / "pulled"
pulled_dir.mkdir(parents=True, exist_ok=True)

# Now you can safely download the file
zip_path = pulled_dir / "callreports_1976_2020_WRDS.dta.zip"

In [67]:
url = "https://pages.stern.nyu.edu/~pschnabl/research/callreports_1976_2020_WRDS.dta.zip"
#zip_path = DATA_DIR / "pulled" / "callreports_1976_2020_WRDS.dta.zip"
urllib.request.urlretrieve(url, zip_path)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(DATA_DIR / "pulled")

# Load the file into a pandas dataframe
data_path = DATA_DIR / "pulled" / "callreports_1976_2020_WRDS.dta"
df = pd.read_stata(data_path)

columns_to_convert_to_int = [
    'rssdid',
    'chartertype',
    'cert',
    'bhcid',
    'date',
    'year',
    'month',
    'quarter',
    'day',
]
df[columns_to_convert_to_int] = df[columns_to_convert_to_int].fillna(99999).astype('int')
df[columns_to_convert_to_int] = df[columns_to_convert_to_int].astype('str')
df[columns_to_convert_to_int] = df[columns_to_convert_to_int].replace('99999', pd.NA)

# df[columns_to_convert_to_int].info()
# df[columns_to_convert_to_int].head()
# df[columns_to_convert_to_int].isna().sum()

# Save the dataframe as a parquet file
parquet_path = DATA_DIR / "pulled" / "nyu_call_report.parquet"
df.to_parquet(parquet_path)

In [68]:
parquet_path = DATA_DIR / "pulled" / "nyu_call_report.parquet"
df = pd.read_parquet(parquet_path)
df

,rssdid,chartertype,cert,bhcid,date,name,assets,reloans,cash,persloans,...,intincnet,nonintexp,nonintinc,operinc,salaries,domdepservicecharges,tradingrevenue,intincreloans1to4fam,intexptimedeple250k,intexptimedepge250k
0,28,200,26606,0,19860331,FIRST ATLANTIC BK,2779.0,0.0,1395.0,23.0,...,21.0,52.0,0.0,22.0,26.0,0.0,NaN,NaN,NaN,NaN
1,28,200,26606,0,19860630,FIRST ATLANTIC BK,4322.0,808.0,153.0,820.0,...,46.0,94.0,3.0,75.0,44.0,2.0,NaN,NaN,NaN,NaN
2,28,200,26606,0,19860930,FIRST ATLANTIC BK,5118.0,1406.0,601.0,1063.0,...,60.0,83.0,10.0,110.0,44.0,7.0,NaN,NaN,NaN,NaN
3,28,200,26606,0,19861231,FIRST ATLANTIC BK,5742.0,1701.0,292.0,995.0,...,65.0,82.0,11.0,122.0,43.0,9.0,NaN,NaN,NaN,NaN
4,28,200,26606,0,19870331,FIRST ATLANTIC BK,7737.0,2451.0,701.0,1179.0,...,73.0,90.0,7.0,141.0,48.0,5.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919805,5427684,200,59182,0,20191231,LOYAL TR BK,25679.0,NaN,23416.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919806,5427684,200,59182,0,20200331,LOYAL TR BK,30489.0,NaN,14296.0,NaN,...,120.0,1070.0,1.0,140.0,745.0,1.0,NaN,0.0,5.0,12.0
1919807,5448915,200,59176,0,20191231,MILLYARD BK,19968.0,NaN,2095.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1919808,5448915,200,59176,0,20200331,MILLYARD BK,23524.0,NaN,4008.0,NaN,...,70.0,659.0,1.0,71.0,422.0,0.0,0.0,4.0,0.0,0.0


In [69]:
entity_name_df = df[df['date'] == '20200331']
entity_name_df

,rssdid,chartertype,cert,bhcid,date,name,assets,reloans,cash,persloans,...,intincnet,nonintexp,nonintinc,operinc,salaries,domdepservicecharges,tradingrevenue,intincreloans1to4fam,intexptimedeple250k,intexptimedepge250k
207,37,200,10057,0,20200331,BANK OF HANCOCK CTY,76416.0,NaN,8415.0,NaN,...,630.0,614.0,67.0,789.0,342.0,33.0,NaN,177.0,76.0,5.0
581,242,200,3850,3088643,20200331,FIRST CMNTY BK XENIA FLORA,43297.0,NaN,4526.0,NaN,...,366.0,230.0,41.0,486.0,151.0,23.0,NaN,124.0,30.0,6.0
744,279,300,28868,3619720,20200331,MINEOLA CMNTY BK SSB,269899.0,NaN,21867.0,NaN,...,1970.0,2018.0,369.0,3004.0,1175.0,142.0,NaN,1606.0,304.0,37.0
921,354,200,14083,0,20200331,BISON ST BK,13722.0,NaN,1224.0,NaN,...,156.0,280.0,91.0,262.0,157.0,9.0,NaN,71.0,9.0,0.0
1224,457,200,10202,1127016,20200331,LOWRY ST BK,55556.0,NaN,6769.0,NaN,...,585.0,388.0,33.0,738.0,223.0,21.0,NaN,246.0,43.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1919801,5401273,200,59178,0,20200331,TANDEM BK,43223.0,NaN,14637.0,NaN,...,231.0,826.0,70.0,338.0,565.0,1.0,NaN,31.0,3.0,1.0
1919804,5412457,200,59117,0,20200331,LEXICON BK,58541.0,NaN,8552.0,NaN,...,389.0,965.0,5.0,469.0,538.0,0.0,NaN,43.0,26.0,2.0
1919806,5427684,200,59182,0,20200331,LOYAL TR BK,30489.0,NaN,14296.0,NaN,...,120.0,1070.0,1.0,140.0,745.0,1.0,NaN,0.0,5.0,12.0
1919808,5448915,200,59176,0,20200331,MILLYARD BK,23524.0,NaN,4008.0,NaN,...,70.0,659.0,1.0,71.0,422.0,0.0,0.0,4.0,0.0,0.0


In [70]:
db = wrds.Connection(wrds_username=WRDS_USERNAME)

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [71]:
db.list_libraries()

['aha_sample',
 'ahasamp',
 'audit',
 'audit_acct_os',
 'audit_audit_comp',
 'audit_common',
 'audit_corp_legal',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'block',
 'block_all',
 'boardex_trial',
 'boardsmp',
 'bvd',
 'bvd_ama_large',
 'bvd_ama_medium',
 'bvd_ama_small',
 'bvd_ama_verylarge',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_large',
 'bvd_orbis_medium',
 'bvd_orbis_small',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'ciq',
 'ciq_capstrct',
 'ciq_common',
 'ciq_ratings',
 'ciq_transactions',
 'ciq_transcripts',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_emdb_daily',
 'comp_emdb_monthly',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 

In [72]:
db.list_tables(library='bank_all')

['wrds_bank_reg_vars',
 'wrds_call_rcfa_1',
 'wrds_call_rcfd_1',
 'wrds_call_rcfd_2',
 'wrds_call_rcfn_1',
 'wrds_call_rcfw_1',
 'wrds_call_rcoa_1',
 'wrds_call_rcon_1',
 'wrds_call_rcon_2',
 'wrds_call_rcow_1',
 'wrds_call_riad_1',
 'wrds_call_te_1',
 'wrds_holding_bhck_1',
 'wrds_holding_bhck_2',
 'wrds_holding_other_1']

#### not working on my computer……

In [73]:
query = f"""
    SELECT 
        
        rcfd_1.rssd9001, rcfd_1.rssd9050, rcfd_1.rssd9999,
        rcfd_1.rcfd0010,rcfd_1.rcfd1773,rcfd_1.rcfdg301,rcfd_1.rcfdg303,
        rcfd_1.rcfdg305,rcfd_1.rcfdg307,rcfd_1.rcfdg309,rcfd_1.rcfdg311,
        rcfd_1.rcfdg313,rcfd_1.rcfdg315,rcfd_1.rcfdg317,rcfd_1.rcfdg319,
        rcfd_1.rcfdg321,rcfd_1.rcfdg323,rcfd_1.rcfdk143,rcfd_1.rcfdk145,
        rcfd_1.rcfdk147,rcfd_1.rcfdk149, rcfd_1.rcfdk151,rcfd_1.rcfdk153,
        rcfd_1.rcfdk155,rcfd_1.rcfdk157, rcfd_1.rcfdc988,rcfd_1.rcfdc027,
        rcfd_1.RCFD1738,rcfd_1.RCFD1741, rcfd_1.RCFD1743,rcfd_1.RCFD1746,
        rcfd_1.rcfdf158,rcfd_1.rcfdf159, rcfd_1.rcfd5367,rcfd_1.rcfd5368,
        rcfd_1.rcfdf160,rcfd_1.rcfdf161, rcfd_1.rcfd1590,rcfd_1.rcfd1763,
        rcfd_1.rcfd1764,rcfd_1.rcfdb538, rcfd_1.rcfdb539,rcfd_1.rcfdk137,
        rcfd_1.rcfdk137,rcfd_1.rcfdk207,

        rcfd_2.rcfd1771,rcfd_2.rcfd0213,rcfd_2.rcfd1287,rcfd_2.rcfd2122,
        rcfd_2.rcfd1420,rcfd_2.rcfd1797,rcfd_2.rcfd1460,rcfd_2.rcfdb989,

        rcon_1.rconj454,rcon_1.rconb987

    FROM bank_all.wrds_call_rcfd_1 AS rcfd_1
    LEFT JOIN 
        bank_all.wrds_call_rcfd_2 as rcfd_2
    ON 
        rcfd_1.rssd9050 = rcfd_2.rssd9050 AND
        rcfd_1.rssd9001 = rcfd_2.rssd9001 AND
        rcfd_1.rssd9999 <= rcfd_2.rssd9999 AND
        rcfd_2.rssd9999 <= rcfd_1.rssd9999
        
    LEFT JOIN 
        bank_all.wrds_call_rcon_1 as rcon_1
    ON 
        rcfd_1.rssd9050 = rcon_1.rssd9050 AND
        rcfd_1.rssd9001 = rcon_1.rssd9001 AND
        rcfd_1.rssd9999 <= rcon_1.rssd9999 AND
        rcfd_2.rssd9999 <= rcon_1.rssd9999
    WHERE 
        rcfd_1.rssd9999  = '{END_DATE}' 
    """

### SQL to acquire data from WRDS

In [74]:
rcon_1_query = f"""
    SELECT 
        rcon_1.rssd9999, rcon_1.rssd9001, 
        rcon_1.RCON0071, rcon_1.RCON1773, rcon_1.RCONHT55,
        rcon_1.RCONHT57, rcon_1.RCONG309, rcon_1.RCONG311,
        rcon_1.RCONG313, rcon_1.RCONG315, rcon_1.RCONG317,
        rcon_1.RCONG319, rcon_1.RCONG321, rcon_1.RCONG323,
        rcon_1.RCONK143, rcon_1.RCONK145, rcon_1.RCONK147,
        rcon_1.RCONK149, rcon_1.RCONK151, rcon_1.RCONK153,
        rcon_1.RCONK155, rcon_1.RCONK157, rcon_1.RCONC988,
        rcon_1.RCONC027, rcon_1.RCONHT59, rcon_1.RCONHT61,
        rcon_1.RCON1743, rcon_1.RCON1746, rcon_1.RCONF158,
        rcon_1.RCONF159, rcon_1.RCON5367, rcon_1.RCON5368,
        rcon_1.RCONF160, rcon_1.RCONF161, rcon_1.RCON1590,
        rcon_1.RCON1766, rcon_1.RCONB538, rcon_1.RCONK137,
        rcon_1.RCONK207, rcon_1.rconj454, rcon_1.RCONJ454,
        
       
        rcon_1.RCONB987, rcon_1.RCONJ451,

        rcon_1.rconmt91, rcon_1.rconmt87, rcon_1.rconhk14, rcon_1.rconhk15,
        rcon_1.rconb993, rcon_1.rcon3230

   
    FROM 
        bank_all.wrds_call_rcon_1 AS rcon_1
    
    
    WHERE 
        rcon_1.rssd9999  = '{END_DATE}' 
    """
rcon_2_query = f"""
    SELECT 
        rcon_2.rssd9001,rcon_2.rcon0081, rcon_2.rcon1771, rcon_2.rcon0213, 
        rcon_2.rcon1287, rcon_2.rcon1738, rcon_2.rcon1741, rcon_2.rcon2122,
        rcon_2.rcon1420, rcon_2.rcon1797, rcon_2.rcon1460, rcon_2.rconb539,
        rcon_2.rconj464, rcon_2.rconb989,

        rcon_2.rcon2200, rcon_2.rconhk05, rcon_2.rconj474, rcon_2.rconb995,
        rcon_2.rconk222, rcon_2.rcon2948, rcon_2.rcon2930, rcon_2.rcong105,
        rcon_2.rcon3838, rcon_2.rcon3632, rcon_2.rcon2170


    FROM 
        bank_all.wrds_call_rcon_2 AS rcon_2
    
    
    WHERE 
        rcon_2.rssd9999  = '{END_DATE}' 
    """

In [75]:
rcfd_1_query = f"""
    SELECT 
        rcfd_1.rssd9001, rcfd_1.rssd9999,
        rcfd_1.rcfd0010,rcfd_1.rcfd1773,rcfd_1.rcfdg301,rcfd_1.rcfdg303,
        rcfd_1.rcfdg305,rcfd_1.rcfdg307,rcfd_1.rcfdg309,rcfd_1.rcfdg311,
        rcfd_1.rcfdg313,rcfd_1.rcfdg315,rcfd_1.rcfdg317,rcfd_1.rcfdg319,
        rcfd_1.rcfdg321,rcfd_1.rcfdg323,rcfd_1.rcfdk143,rcfd_1.rcfdk145,
        rcfd_1.rcfdk147,rcfd_1.rcfdk149, rcfd_1.rcfdk151,rcfd_1.rcfdk153,
        rcfd_1.rcfdk155,rcfd_1.rcfdk157, rcfd_1.rcfdc988,rcfd_1.rcfdc027,
        rcfd_1.RCFD1738,rcfd_1.RCFD1741, rcfd_1.RCFD1743,rcfd_1.RCFD1746,
        rcfd_1.rcfdf158,rcfd_1.rcfdf159, rcfd_1.rcfd5367,rcfd_1.rcfd5368,
        rcfd_1.rcfdf160,rcfd_1.rcfdf161, rcfd_1.rcfd1590,rcfd_1.rcfd1763,
        rcfd_1.rcfd1764,rcfd_1.rcfdb538, rcfd_1.rcfdb539,rcfd_1.rcfdk137,
        rcfd_1.rcfdk137,rcfd_1.rcfdk207,

        rcfd_1.rcfd2930,rcfd_1.rcfd3230

       

  
       

    FROM 
        bank_all.wrds_call_rcfd_1 AS rcfd_1
    
    
    WHERE 
        rcfd_1.rssd9999  = '{END_DATE}' 
    """
rcfd_2_query = f"""
    SELECT 
        rcfd_2.rssd9001, 
        rcfd_2.rcfd1771, rcfd_2.rcfd0213, rcfd_2.rcfd1287, rcfd_2.rcfd2122,
        rcfd_2.rcfd1420, rcfd_2.rcfd1797, rcfd_2.rcfd1460, rcfd_2.rcfdb989,

        rcfd_2.rcfd2948, rcfd_2.rcfdg105, rcfd_2.rcfd3838, rcfd_2.rcfd3632,
        rcfd_2.rcfd2170

    FROM 
        bank_all.wrds_call_rcfd_2 AS rcfd_2
    
    
    WHERE 
        rcfd_2.rssd9999  = '{END_DATE}' 
    """

In [76]:
holding_other_query = f"""
    SELECT 
        other_1.rssd9001, other_1.rssd9999,other_1.rssd9010

    FROM 
        bank_all.wrds_holding_other_1 AS other_1
    
    
    WHERE 
        other_1.rssd9999  = '{END_DATE}' 
    """

In [77]:
rcfn_query = f"""
    SELECT 
        rcfn_1.rssd9001, rcfn_1.rcfn2200

    FROM 
        bank_all.wrds_call_rcfn_1 AS rcfn_1
    
    
    WHERE 
        rcfn_1.rssd9999  = '{END_DATE}' 
    """

### Most banks have Domestic Offices Only

In [78]:
rcon_1_df = db.raw_sql(rcon_1_query)
rcon_1_df = rcon_1_df.sort_values(by = 'rssd9001').set_index('rssd9001')

rcon_2_df = db.raw_sql(rcon_2_query)
rcon_2_df = rcon_2_df.sort_values(by = 'rssd9001').set_index('rssd9001')

rcon_df = pd.merge(rcon_1_df, rcon_2_df, left_index=True, right_index=True, how='inner')
rcon_df.head()

,rssd9999,rcon0071,rcon1773,rconht55,rconht57,rcong309,rcong311,rcong313,rcong315,rcong317,...,rconhk05,rconj474,rconb995,rconk222,rcon2948,rcon2930,rcong105,rcon3838,rcon3632,rcon2170
rssd9001,,,,,,,,,,,,,,,,,,,,,
37,2022-03-31,10392.0,53347.0,0.0,6877.0,0.0,0.0,0.0,0.0,0.0,...,0,1718,0.0,1035,73591.0,96.0,14251.0,0.0,11761.0,87842.0
242,2022-03-31,2945.0,18384.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,4153,0.0,3624,49360.0,16.0,3876.0,0.0,4947.0,53236.0
279,2022-03-31,5335.0,63978.0,28067.0,17274.0,0.0,0.0,0.0,10287.0,0.0,...,0,14346,0.0,9473,324505.0,2315.0,44370.0,0.0,32277.0,368875.0
354,2022-03-31,500.0,745.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,...,0,261,0.0,261,21004.0,132.0,4073.0,0.0,698.0,25077.0
457,2022-03-31,1991.0,1431.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,544,0.0,544,60226.0,92.0,5110.0,0.0,1188.0,65336.0


In [79]:
rcfd_1_df = db.raw_sql(rcfd_1_query)
rcfd_1_df = rcfd_1_df.sort_values(by = 'rssd9001').set_index('rssd9001')
rcfd_2_df = db.raw_sql(rcfd_2_query)
rcfd_2_df = rcfd_2_df.sort_values(by = 'rssd9001').set_index('rssd9001')
rcfd_df = pd.merge(rcfd_1_df, rcfd_2_df, left_index=True, right_index=True, how='inner')
rcfd_df.head()

,rssd9999,rcfd0010,rcfd1773,rcfdg301,rcfdg303,rcfdg305,rcfdg307,rcfdg309,rcfdg311,rcfdg313,...,rcfd2122,rcfd1420,rcfd1797,rcfd1460,rcfdb989,rcfd2948,rcfdg105,rcfd3838,rcfd3632,rcfd2170
rssd9001,,,,,,,,,,,,,,,,,,,,,
37,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
354,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
457,2022-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Get the dataframe contains entity name

In [122]:
name_df = entity_name_df.sort_values(by = 'bhcid')
name_df.set_index('bhcid', inplace=True)
name_df

,rssdid,chartertype,cert,date,name,assets,reloans,cash,persloans,agloans,...,intincnet,nonintexp,nonintinc,operinc,salaries,domdepservicecharges,tradingrevenue,intincreloans1to4fam,intexptimedeple250k,intexptimedepge250k
bhcid,,,,,,,,,,,,,,,,,,,,,
0,37,200,10057,20200331,BANK OF HANCOCK CTY,76416.0,NaN,8415.0,NaN,33.0,...,630.0,614.0,67.0,789.0,342.0,33.0,NaN,177.0,76.0,5.0
0,645577,300,31198,20200331,GREENEVILLE FED BK FSB,149920.0,NaN,15824.0,NaN,628.0,...,1415.0,1361.0,115.0,1858.0,805.0,44.0,NaN,771.0,277.0,10.0
0,645540,200,5796,20200331,EARLHAM SVG BK,319995.0,NaN,7794.0,NaN,16469.0,...,2375.0,2332.0,316.0,3159.0,1370.0,161.0,NaN,504.0,292.0,31.0
0,644954,300,35510,20200331,DAVIDSON TC,10552.0,NaN,0.0,NaN,0.0,...,44.0,1452.0,1839.0,1883.0,739.0,0.0,NaN,0.0,0.0,0.0
0,644141,200,5892,20200331,KALAMAZOO CTY ST BK,98722.0,NaN,8699.0,NaN,1724.0,...,864.0,825.0,82.0,973.0,435.0,33.0,NaN,202.0,11.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5422531,883959,200,11526,20200331,BANK OF KREMLIN,322299.0,NaN,13889.0,NaN,81098.0,...,3177.0,1894.0,324.0,4221.0,1167.0,135.0,NaN,293.0,436.0,136.0
5423547,927479,300,29941,20200331,BOGOTA SVG BK,708777.0,NaN,38421.0,NaN,0.0,...,3091.0,5072.0,115.0,6038.0,1295.0,5.0,NaN,3527.0,1646.0,518.0
5425653,339773,320,26483,20200331,NEEDHAM BK,2495379.0,NaN,100498.0,NaN,0.0,...,17369.0,13138.0,2479.0,28284.0,8691.0,130.0,0.0,9551.0,3834.0,2001.0


In [81]:
holding_company_data = db.raw_sql(holding_other_query)
holding_company_data['rssd9001'] = holding_company_data['rssd9001'].astype('int64').astype(str)
holding_company_data.set_index('rssd9001', inplace= True)
holding_company_data

,rssd9999,rssd9010
rssd9001,,
1020180,2022-03-31,BREMER FNCL CORP
1020201,2022-03-31,HSBC USA
1026632,2022-03-31,CHARLES SCHWAB CORP
1037003,2022-03-31,M&T BK CORP
1048773,2022-03-31,VALLEY NAT BC
...,...,...
4529394,2022-03-31,FORD ULTIMATE MGMT II LLC
4529406,2022-03-31,FORD FAM INV LP
4933272,2022-03-31,CAPSTAR FNCL HOLDS


In [123]:
name_df['higher_holder'] = np.nan
name_df.loc[name_df.index.isin(holding_company_data.index),['higher_holder']] = holding_company_data['rssd9010']
name_df['rssdid'] = name_df['rssdid'].astype('int64')
name_df = name_df.rename(columns={'rssdid': 'rssd9001'})
name_df = name_df.reset_index().set_index('rssd9001')
name_df

C:\Users\zwr\AppData\Local\Temp\ipykernel_18544\1976720277.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['CENTRAL PACIFIC FC', 'BANK OF HI CORP', 'WESTAMERICA BC', 'FIRST HAWAIIAN', 'CHARLES SCHWAB CORP', 'CHARLES SCHWAB CORP', 'CHARLES SCHWAB CORP', 'FREMONT BC', 'CVB FC', 'WTB FC', 'TRICO BSHRS', 'SVB FNCL GRP', 'FINANCIAL INST', 'M&T BK CORP', 'M&T BK CORP', 'JPMORGAN CHASE & CO', 'JPMORGAN CHASE & CO', 'JPMORGAN CHASE & CO', 'TRUSTCO BC NY', 'CONNECTONE BC', 'VALLEY NAT BC', 'ARROW FC', 'ARROW FC', 'COMMUNITY BK SYS', 'FIRST OF LONG ISLAND CORP', 'COMMERCE BSHRS', 'UMB FC', 'UMB FC', 'VALLEY VIEW BSHRS', 'PINNACLE BC', 'PINNACLE BC', 'PINNACLE BC', 'PINNACLE BC', 'FARMERS & MRCH INV', 'SPEND LIFE WISELY CO', 'AMERICAN NAT CORP', 'FIRSTBANK HC', 'ALPINE BK OF CO', 'INTRUST FC', 'LAURITZEN CORP', 'FIRSTSUN CAP BC', 'KEYCORP', 'KEYCORP', 'KEYCORP', 'HUNTINGTON BSHRS', 'PNC FNCL SVC GROUP', 'PNC FNCL SVC GRO

,bhcid,chartertype,cert,date,name,assets,reloans,cash,persloans,agloans,...,nonintexp,nonintinc,operinc,salaries,domdepservicecharges,tradingrevenue,intincreloans1to4fam,intexptimedeple250k,intexptimedepge250k,higher_holder
rssd9001,,,,,,,,,,,,,,,,,,,,,
37,0,200,10057,20200331,BANK OF HANCOCK CTY,76416.0,NaN,8415.0,NaN,33.0,...,614.0,67.0,789.0,342.0,33.0,NaN,177.0,76.0,5.0,NaN
645577,0,300,31198,20200331,GREENEVILLE FED BK FSB,149920.0,NaN,15824.0,NaN,628.0,...,1361.0,115.0,1858.0,805.0,44.0,NaN,771.0,277.0,10.0,NaN
645540,0,200,5796,20200331,EARLHAM SVG BK,319995.0,NaN,7794.0,NaN,16469.0,...,2332.0,316.0,3159.0,1370.0,161.0,NaN,504.0,292.0,31.0,NaN
644954,0,300,35510,20200331,DAVIDSON TC,10552.0,NaN,0.0,NaN,0.0,...,1452.0,1839.0,1883.0,739.0,0.0,NaN,0.0,0.0,0.0,NaN
644141,0,200,5892,20200331,KALAMAZOO CTY ST BK,98722.0,NaN,8699.0,NaN,1724.0,...,825.0,82.0,973.0,435.0,33.0,NaN,202.0,11.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883959,5422531,200,11526,20200331,BANK OF KREMLIN,322299.0,NaN,13889.0,NaN,81098.0,...,1894.0,324.0,4221.0,1167.0,135.0,NaN,293.0,436.0,136.0,NaN
927479,5423547,300,29941,20200331,BOGOTA SVG BK,708777.0,NaN,38421.0,NaN,0.0,...,5072.0,115.0,6038.0,1295.0,5.0,NaN,3527.0,1646.0,518.0,NaN
339773,5425653,320,26483,20200331,NEEDHAM BK,2495379.0,NaN,100498.0,NaN,0.0,...,13138.0,2479.0,28284.0,8691.0,130.0,0.0,9551.0,3834.0,2001.0,NaN


In [83]:
rcfn_df = db.raw_sql(rcfn_query)

# The dataframe I used has different rssd id compared to other data sources

### assign groups to code

In [84]:
global_rmbs = ['rcfdg301','rcfdg303','rcfdg305','rcfdg307','rcfdg309','rcfdg311',
               'rcfdg313','rcfdg315','rcfdg317','rcfdg319','rcfdg321','rcfdg323']
global_cmbs = ['rcfdk143','rcfdk145','rcfdk147','rcfdk149','rcfdk151','rcfdk153',
               'rcfdk157']
global_abs = ['rcfdc988','rcfdc027']
global_other = ['rcfd1738','rcfd1741','rcfd1743','rcfd1746']
global_rs_loan = ['rcfdf158','rcfdf159','rcfd1420','rcfd1420','rcfd1797','rcfd5367','rcfd5368','rcfd1460','rcfdf160','rcfdf161']
global_rs_residential_loan = ['rcfd1420', 'rcfd1797','rcfd5367','rcfd5368','rcfd1460']
global_rs_commerical_loan = ['rcfdf160','rcfdf161']
global_rs_other_loan = ['rcfdf158','rcfdf159']
global_ci_loan = ['rcfd1763','rcfd1764']
global_consumer_loan = ['rcfdb538','rcfdb539','rcfdk137','rcfdk207']

In [85]:
domestic_cash = ['rcon0081','rcon0071']
domestic_total = ['rcon1771','rcon1773']
domestic_treasury = ['rcon0213','rcon1287']
domestic_rmbs = ['rconht55','rconht57','rcong309','rcong311',
               'rcong313','rcong315','rcong317','rcong319','rcong321','rcong323']
domestic_cmbs = ['rconk143','rconk145','rconk147','rconk149','rconk151','rconk153',
               'rconk157']
domestic_abs = ['rconc988','rconc027','rconht59','rconht61']
domestic_other = ['rcon1738','rcon1741','rcon1743','rcon1746']
domestic_rs_loan = ['rconf158','rconf159','rcon1420','rcon1420','rcon1797','rcon5367','rcon5368','rcon1460','rconf160','rconf161']
domestic_rs_residential_loan = ['rcon1420','rcon1797','rcon5367','rcon5368','rcon1460']
domestic_rs_commerical_loan = ['rconf160','rconf161']
domestic_rs_other_loan = ['rconf158','rconf159']
domestic_ci_loan = ['rcon1766']
domestic_consumer_loan = ['rconb538','rconb539','rconk137','rconk207']
domestic_non_rep_loan = ['rconj454','rconj464','rconj451']

In [86]:
insured_deposit = ['rconhk05','rconmt91','rconmt87']
uninsured_long = ['rconhk14','rconhk15']

### rearrange output names

In [87]:
rcfd_data = pd.DataFrame(index= rcfd_df.index)
rcfd_data['Total Asset'] = rcfd_df['rcfd2170']
rcfd_data['cash'] = rcfd_df['rcfd0010']
rcfd_data['security_total'] = rcfd_df['rcfd1771'] + rcfd_df['rcfd1773']
rcfd_data['security_treasury'] = rcfd_df['rcfd0213'] + rcfd_df['rcfd1287']
rcfd_data['security_rmbs'] = rcfd_df[global_rmbs].sum(axis=1)
rcfd_data['security_cmbs'] = rcfd_df[global_cmbs].sum(axis=1)
rcfd_data['security_abs'] = rcfd_df[global_abs].sum(axis=1)
rcfd_data['security_other'] = rcfd_df[global_other].sum(axis=1)
rcfd_data['Total_Loan'] = rcfd_df['rcfd2122']
rcfd_data['Real_Estate_Loan'] = rcfd_df[global_rs_loan].sum(axis=1)
rcfd_data['Residential_Mortgage'] = rcfd_df[global_rs_residential_loan].sum(axis=1)
rcfd_data['Commerical_Mortgage'] = rcfd_df[global_rs_commerical_loan].sum(axis=1)
rcfd_data['Other_Real_Estate_Mortgage'] = rcfd_df[global_rs_other_loan].sum(axis=1)
rcfd_data['Agri_Loan'] = rcfd_df['rcfd1590']
rcfd_data['Comm_Indu_Loan'] = rcfd_df[global_ci_loan].sum(axis=1)
rcfd_data['Consumer_Loan'] = rcfd_df[global_consumer_loan].sum(axis=1)
rcfd_data['Non_Rep_Loan'] =  np.nan
rcfd_data['Fed_Fund_Sold'] = rcon_df['rconb987']
rcfd_data['Reverse_Repo'] = rcfd_df['rcfdb989']
rcfd_data.head()

,Total Asset,cash,security_total,security_treasury,security_rmbs,security_cmbs,security_abs,security_other,Total_Loan,Real_Estate_Loan,Residential_Mortgage,Commerical_Mortgage,Other_Real_Estate_Mortgage,Agri_Loan,Comm_Indu_Loan,Consumer_Loan,Non_Rep_Loan,Fed_Fund_Sold,Reverse_Repo
rssd9001,,,,,,,,,,,,,,,,,,,
37,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0,NaN
242,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,3049,NaN
279,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,19742,NaN
354,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,6870,NaN
457,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,325,NaN


In [88]:
rcon_data = pd.DataFrame(index= rcon_df.index)
rcon_data['Total Asset'] = rcon_df['rcon2170']
rcon_data['cash'] = rcon_df[domestic_cash].sum(axis = 1)
rcon_data['security_total'] = rcon_df[domestic_total].sum(axis = 1)
rcon_data['security_treasury'] = rcon_df[domestic_treasury].sum(axis = 1)
rcon_data['security_rmbs'] = rcon_df[domestic_rmbs].sum(axis=1)
rcon_data['security_cmbs'] = rcon_df[domestic_cmbs].sum(axis=1)
rcon_data['security_abs'] = rcon_df[domestic_abs].sum(axis=1)
rcon_data['security_other'] = rcon_df[domestic_other].sum(axis=1)
rcon_data['Total_Loan'] = rcon_df['rcon2122']
rcon_data['Real_Estate_Loan'] = rcon_df[domestic_rs_loan].sum(axis=1)
rcon_data['Residential_Mortgage'] = rcon_df[domestic_rs_residential_loan].sum(axis=1)
rcon_data['Commerical_Mortgage'] = rcon_df[domestic_rs_commerical_loan].sum(axis=1)
rcon_data['Other_Real_Estate_Mortgage'] = rcon_df[domestic_rs_other_loan].sum(axis=1)
rcon_data['Agri_Loan'] = rcon_df['rcon1590']
rcon_data['Comm_Indu_Loan'] = rcon_df[domestic_ci_loan].sum(axis=1)
rcon_data['Consumer_Loan'] = rcon_df[domestic_consumer_loan].sum(axis=1)
rcon_data['Non_Rep_Loan'] =  rcon_df[domestic_non_rep_loan].sum(axis=1)
rcon_data['Fed_Fund_Sold'] = rcon_df['rconb987']
rcon_data['Reverse_Repo'] = rcon_df['rconb989']
rcon_data.head()

,Total Asset,cash,security_total,security_treasury,security_rmbs,security_cmbs,security_abs,security_other,Total_Loan,Real_Estate_Loan,Residential_Mortgage,Commerical_Mortgage,Other_Real_Estate_Mortgage,Agri_Loan,Comm_Indu_Loan,Consumer_Loan,Non_Rep_Loan,Fed_Fund_Sold,Reverse_Repo
rssd9001,,,,,,,,,,,,,,,,,,,
37,87842.0,12640.0,53347.0,0.0,6877.0,0.0,0.0,0.0,20854.0,10869,8917,866,229,39,4592.0,4784,27.0,0,0.0
242,53236.0,5146.0,18384.0,2563.0,0.0,0.0,0.0,411.0,25865.0,20989,14321,3910,0,1415,1976.0,3557,13.0,3049,0.0
279,368875.0,11163.0,94061.0,18162.0,55628.0,0.0,0.0,4363.0,226173.0,220122,161682,33226,19026,211,6745.0,4041,87.0,19742,0.0
354,25077.0,1112.0,745.0,192.0,58.0,0.0,0.0,0.0,16177.0,13200,12068,492,132,1209,1155.0,791,330.0,6870,0.0
457,65336.0,3441.0,1431.0,1431.0,0.0,0.0,0.0,0.0,58988.0,43929,27909,4861,4259,6061,9493.0,4763,135.0,325,0.0


### Special adjustment for Non_Rep_Loan since both global and domestic banks have the same code

In [127]:
bank_asset = pd.merge(rcfd_data, rcon_data, left_index=True, right_index=True, how='outer', suffixes=('', '_df2'))
replace_index = bank_asset[bank_asset['cash'].isna()].index

bank_asset.loc[replace_index, bank_asset.columns[:19]] = bank_asset.loc[replace_index, bank_asset.columns[19:]].values
bank_asset.drop(columns='Non_Rep_Loan', inplace= True)
bank_asset.rename(columns = {'Non_Rep_Loan_df2':'Non_Rep_Loan'}, inplace= True)
columns_to_drop = [col for col in bank_asset.columns if '_df2' in col]
bank_asset.drop(columns=columns_to_drop, inplace= True)

bank_asset[['bhcid','higher_holder'] ]= np.nan

bank_asset.loc[bank_asset.index.isin(name_df.index),['bhcid','higher_holder']] =  name_df[['bhcid','higher_holder']]
bank_asset

C:\Users\zwr\AppData\Local\Temp\ipykernel_18544\3900695867.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '3088643' '3619720' ... '0' '0' '0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bank_asset.loc[bank_asset.index.isin(name_df.index),['bhcid','higher_holder']] =  name_df[['bhcid','higher_holder']]
C:\Users\zwr\AppData\Local\Temp\ipykernel_18544\3900695867.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bank_asset.loc[bank_asset.index.isin(name_df.index),['bhcid','higher_holder']] =  name_df[['bhcid','higher_holder']]


,Total Asset,cash,security_total,security_treasury,security_rmbs,security_cmbs,security_abs,security_other,Total_Loan,Real_Estate_Loan,...,Commerical_Mortgage,Other_Real_Estate_Mortgage,Agri_Loan,Comm_Indu_Loan,Consumer_Loan,Fed_Fund_Sold,Reverse_Repo,Non_Rep_Loan,bhcid,higher_holder
rssd9001,,,,,,,,,,,,,,,,,,,,,
37,87842.0,12640.0,53347.0,0.0,6877.0,0.0,0.0,0.0,20854.0,10869.0,...,866.0,229.0,39.0,4592.0,4784.0,0,0.0,27.0,0,NaN
242,53236.0,5146.0,18384.0,2563.0,0.0,0.0,0.0,411.0,25865.0,20989.0,...,3910.0,0.0,1415.0,1976.0,3557.0,3049,0.0,13.0,3088643,NaN
279,368875.0,11163.0,94061.0,18162.0,55628.0,0.0,0.0,4363.0,226173.0,220122.0,...,33226.0,19026.0,211.0,6745.0,4041.0,19742,0.0,87.0,3619720,NaN
354,25077.0,1112.0,745.0,192.0,58.0,0.0,0.0,0.0,16177.0,13200.0,...,492.0,132.0,1209.0,1155.0,791.0,6870,0.0,330.0,0,NaN
457,65336.0,3441.0,1431.0,1431.0,0.0,0.0,0.0,0.0,58988.0,43929.0,...,4861.0,4259.0,6061.0,9493.0,4763.0,325,0.0,135.0,1127016,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5620869,95052.0,25906.0,0.0,0.0,0.0,0.0,0.0,0.0,59837.0,59837.0,...,14916.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN
5650923,87152.0,51512.0,9342.0,0.0,6629.0,0.0,0.0,0.0,24447.0,21704.0,...,15805.0,0.0,0.0,2667.0,76.0,0,0.0,0.0,NaN,NaN
5660146,35261.0,30436.0,0.0,0.0,0.0,0.0,0.0,0.0,3113.0,3113.0,...,3113.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,NaN


### LIABILITY

In [92]:
global_liability = pd.DataFrame(index= rcon_df.index)
global_liability['Total Liability'] = rcfd_df['rcfd2948']
global_liability['Domestic Deposit'] = rcon_df['rcon2200']
global_liability['Insured Deposit'] = rcon_df[insured_deposit].sum(axis=1)
global_liability['Uninsured Deposit'] = global_liability['Domestic Deposit'] - global_liability['Insured Deposit']
global_liability['Uninsured Time Deposits'] = rcon_df['rconj474']
global_liability['Uninsured Long-Term Time Deposits'] = rcon_df[uninsured_long].sum(axis=1)
global_liability['Uninsured Short-Term Time Deposits'] = rcon_df['rconk222']
global_liability['Foreign Deposit'] = rcfn_df['rcfn2200']
global_liability['Fed Fund Purchase'] = rcon_df['rconb993']
global_liability['Repo'] = rcon_df['rconb995']
global_liability['Other Liability'] = rcfd_df['rcfd2930']
global_liability['Total Equity'] = rcfd_df['rcfdg105']
global_liability['Common Stock'] = rcfd_df['rcfd3230']
global_liability['Preferred Stock'] = rcfd_df['rcfd3838']
global_liability['Retained Earning'] = rcfd_df['rcfd3632']


In [93]:
domestic_liability = pd.DataFrame(index= rcon_df.index)
domestic_liability['Total Liability'] = rcon_df['rcon2948']
domestic_liability['Domestic Deposit'] = rcon_df['rcon2200']
domestic_liability['Insured Deposit'] = rcon_df[insured_deposit].sum(axis=1)
domestic_liability['Uninsured Deposit'] = domestic_liability['Domestic Deposit'] - domestic_liability['Insured Deposit']
domestic_liability['Uninsured Time Deposits'] = rcon_df['rconj474']
domestic_liability['Uninsured Long-Term Time Deposits'] = rcon_df[uninsured_long].sum(axis=1)
domestic_liability['Uninsured Short-Term Time Deposits'] = rcon_df['rconk222']
domestic_liability['Foreign Deposit'] = rcfn_df['rcfn2200']
domestic_liability['Fed Fund Purchase'] = rcon_df['rconb993']
domestic_liability['Repo'] = rcon_df['rconb995']
domestic_liability['Other Liability'] = rcon_df['rcon2930']
domestic_liability['Total Equity'] = rcon_df['rcong105']
domestic_liability['Common Stock'] = rcon_df['rcon3230']
domestic_liability['Preferred Stock'] = rcon_df['rcon3838']
domestic_liability['Retained Earning'] = rcon_df['rcon3632']

In [129]:
bank_liability = pd.merge(global_liability, domestic_liability, left_index=True, right_index=True, how='outer', suffixes=('', '_df2'))
replace_index = bank_liability[bank_liability['Total Liability'].isna()].index

bank_liability.loc[replace_index, bank_liability.columns[:15]] = bank_liability.loc[replace_index, bank_liability.columns[15:]].values
columns_to_drop = [col for col in bank_liability.columns if '_df2' in col]
bank_liability.drop(columns=columns_to_drop, inplace= True)
bank_liability[['bhcid','higher_holder']] = np.nan
bank_liability.loc[bank_liability.index.isin(name_df.index),['bhcid','higher_holder']] =  name_df[['bhcid','higher_holder']]
bank_liability

C:\Users\zwr\AppData\Local\Temp\ipykernel_18544\175285046.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '3088643' '3619720' ... '0' '0' '0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bank_liability.loc[bank_liability.index.isin(name_df.index),['bhcid','higher_holder']] =  name_df[['bhcid','higher_holder']]
C:\Users\zwr\AppData\Local\Temp\ipykernel_18544\175285046.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  bank_liability.loc[bank_liability.index.isin(name_df.index),['bhcid','higher_holder']] =  name_df[['bhcid','higher_holder']]


,Total Liability,Domestic Deposit,Insured Deposit,Uninsured Deposit,Uninsured Time Deposits,Uninsured Long-Term Time Deposits,Uninsured Short-Term Time Deposits,Foreign Deposit,Fed Fund Purchase,Repo,Other Liability,Total Equity,Common Stock,Preferred Stock,Retained Earning,bhcid,higher_holder
rssd9001,,,,,,,,,,,,,,,,,
37,73591.0,73495.0,0.0,73495.0,1718,683,1035,NaN,0,0.0,96.0,14251.0,1200.0,0.0,11761.0,0,NaN
242,49360.0,44994.0,0.0,44994.0,4153,529,3624,NaN,0,0.0,16.0,3876.0,120.0,0.0,4947.0,3088643,NaN
279,324505.0,295137.0,0.0,295137.0,14346,4874,9473,NaN,0,0.0,2315.0,44370.0,1.0,0.0,32277.0,3619720,NaN
354,21004.0,20772.0,0.0,20772.0,261,0,261,NaN,0,0.0,132.0,4073.0,100.0,0.0,698.0,0,NaN
457,60226.0,52134.0,0.0,52134.0,544,0,544,NaN,0,0.0,92.0,5110.0,60.0,0.0,1188.0,1127016,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5620869,51982.0,43039.0,4755.0,38284.0,0,0,0,NaN,0,0.0,8943.0,43070.0,50834.0,5980.0,-14779.0,NaN,NaN
5650923,45011.0,44905.0,0.0,44905.0,2300,2000,300,NaN,0,0.0,106.0,42141.0,22500.0,0.0,-2369.0,NaN,NaN
5660146,1570.0,1570.0,0.0,1570.0,0,0,0,NaN,0,0.0,0.0,33691.0,100.0,0.0,-30.0,NaN,NaN
